<a href="https://colab.research.google.com/github/ElenaShargina/patterns/blob/master/%D0%9F%D0%BE%D0%B2%D0%B5%D0%B4%D0%B5%D0%BD%D1%87%D0%B5%D1%81%D0%BA%D0%B8%D0%B5%20%D0%BF%D0%B0%D1%82%D1%82%D0%B5%D1%80%D0%BD%D1%8B/Interpreter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Interpreter / Интерпретатор
Для заданного языка определяет представление его грамматики, а также интерпретатор предложений этого языка.

## Пример реализации

Реализуем простой интерпретатор булевых выражений.

<img src='http://feana.ru/wp-content/uploads/2023/05/interpreter-e1685284341688.png'>

In [12]:
# класс контекста, где будут храниться значения именованных переменных
class Context:
    def __init__(self):
        self.values = {}

    def look_up(self, name):
        if name in self.values:
            return self.values[name]
        else:
            raise Exception(f'No variable {name} is assigned in current context.')

    def assign(self, a, b:bool):
        self.values[a.name] = b
        pass
    
    def __str__(self):
        return "Context: \n"+"\n".join([f'{i} = {j}' for i,j in self.values.items()])

# абстрактный класс - булевое выражение
class BooleanExpression:
    def evaluate(self,context:Context):
        pass
    def replace(self, value, a):
        pass
    def copy(self):
        pass

# конкретный класс - булевое выражение _ AND _ с двумя переменными
class AndExpression(BooleanExpression):
    def __init__(self,a:BooleanExpression,b:BooleanExpression):
        self.a = a
        self.b = b

    def __str__(self):
      return f'({str(self.a)}) AND ({str(self.b)})'

    def evaluate(self,context:Context):
        return self.a.evaluate(context) & self.b.evaluate(context)
    
    def evaluate_show(self,context:Context):
        return f'({self.a.evaluate_show(context)}) AND ({self.b.evaluate_show(context)})'

    def copy(self):
        return AndExpression(self.a.copy(), self.b.copy())

    def replace(self, name, z:BooleanExpression):
        return AndExpression(self.a.replace(name,z),self.b.replace(name,z))

# конкретный класс - булевое выражение _ OR _ с двумя переменными
class OrExpression(BooleanExpression):
    def __init__(self,a:BooleanExpression,b:BooleanExpression):
        self.a = a
        self.b = b

    def __str__(self):
      return f'({str(self.a)}) OR ({str(self.b)})'

    def evaluate(self,context:Context):
        return self.a.evaluate(context) or self.b.evaluate(context)
    
    def evaluate_show(self,context:Context):
        return f'({self.a.evaluate_show(context)}) OR ({self.b.evaluate_show(context)})'

    def replace(self, name, z:BooleanExpression):
        return OrExpression(self.a.replace(name,z),self.b.replace(name,z))

# конкретный класс - булевое выражение NOT _ с одной переменной
class NotExpression(BooleanExpression):
    def __init__(self,a:BooleanExpression):
        self.a = a

    def __str__(self):
      return f'NOT ({str(self.a)})'

    def evaluate(self,context:Context):
        return not self.a.evaluate((context))
     
    def evaluate_show(self,context:Context):
        return f'NOT ({self.a.evaluate_show(context)})'

    def replace(self, name, z:BooleanExpression):
        return NotExpression(self.a.replace(name,z))



# класс именованной переменной
# 'A' | 'B' | 'C' | ...
class VariableExpression(BooleanExpression):
    def __init__(self, name):
        self.name = name

    def __str__(self):
        return self.name 

    def evaluate(self,context:Context):
        return context.look_up(self.name)

    def evaluate_show(self,context:Context):
        return str(context.look_up(self.name))

    def copy(self):
        return VariableExpression(self.name)

    def replace(self, name, z:BooleanExpression):
        if (name == self.name):
            return z.copy()
        else:
            return VariableExpression(name)


# Заведем новый конекст
c = Context()
# Зафиксируем две именованные переменные X и Y
x = VariableExpression('X')
y = VariableExpression('Y')
# Запишем выражение Not(X) And Y
exp = AndExpression(NotExpression(x),y)
print(str(exp))

# Подставим на место X значение False
c.assign(x,False)
# Подставим на место Y значение True
c.assign(y,True)

# выведем контекст
print(c)

# Выведем результат подсчета выражения
print(exp.evaluate_show(c))
print(exp.evaluate(c))

# Введем новую именованную переменную Z
z = VariableExpression('Z')
# Запишем новое выражение, где на место Y встанет Z
new_exp = exp.replace('Y',z)
print(str(new_exp))
# Подставим на место Z значение False
c.assign(z,False)
# выведем обновленный контекст
print(c)
# Выведем результат подсчета выражения
print(new_exp.evaluate_show(c))
print(new_exp.evaluate(c))

(NOT (X)) AND (Y)
Context: 
X = False
Y = True
(NOT (False)) AND (True)
True
(NOT (Y)) AND (Z)
Context: 
X = False
Y = True
Z = False
(NOT (True)) AND (False)
False
